In [1]:
!git clone https://github.com/ngoxuanphong/ENV.git
%cd ENV

Cloning into 'ENV'...
remote: Enumerating objects: 1470, done.
remote: Counting objects: 100% (696/696), done.
remote: Compressing objects: 100% (424/424), done.
remote: Total 1470 (delta 294), reused 583 (delta 204), pack-reused 774
Receiving objects: 100% (1470/1470), 67.24 MiB | 17.76 MiB/s, done.
Resolving deltas: 100% (604/604), done.
Updating files: 100% (225/225), done.
/content/ENV


In [2]:

# Check hệ thống

# import Base.MachiKoro.env as env
# from CheckEnv import check_env
# print(check_env(env))

import numpy as np
import random as rd
from numba import njit, jit
import sys, os
from setup import SHORT_PATH
import importlib.util
game_name = 'TLMN'

def add_game_to_syspath(game_name):
    if len(sys.argv) >= 2:
        sys.argv = [sys.argv[0]]
    sys.argv.append(game_name)

def setup_game(game_name):
    spec = importlib.util.spec_from_file_location('env', f"{SHORT_PATH}Base/{game_name}/env.py")
    module = importlib.util.module_from_spec(spec)
    sys.modules[spec.name] = module 
    spec.loader.exec_module(module)
    return module

add_game_to_syspath(game_name)
env = setup_game(game_name)

getActionSize = env.getActionSize
getStateSize = env.getStateSize
getAgentSize = env.getAgentSize

getValidActions = env.getValidActions
getReward = env.getReward
numba_main_2 = env.numba_main_2

@njit()
def bot_lv0(state, perData):
    validActions = getValidActions(state)
    arr_action = np.where(validActions==1)[0]
    idx = np.random.randint(0, arr_action.shape[0])
    return arr_action[idx], perData
__ACTIONS__ = env.__ACTIONS__

In [21]:

#---------------------------------------------------------------------------------------------------------
from numba.typed import List
@njit()
def getCardOnHand(state):
    card_on_hand = state[0:52]
    return np.where(card_on_hand == 1)[0]
@njit()
def checkArrInArr(arr1,arr2):
  # print('arr1,arr2',arr1,arr2)
  arr_return = []
  for i in range(len(arr1)):
    if arr1[i] in arr2:
      arr_return.append(i)
  return np.array(arr_return)
@njit()
def getSpecialCard(state):
    p_cards = state[0:52]
    p_cards = np.where(p_cards == 1)[0]
    p_cards_value = p_cards//4
    policy = np.zeros(52)
    for i in range(52):
        arr_copy = np.copy(p_cards)
        arr_temp = np.where(arr_copy // 4 == i)[0] 
    # check day
    arr_day = np.zeros(11) - 1
    arr_unique = np.unique(p_cards_value)
    for i in range(1,len(arr_unique)):
        if arr_unique[i] == arr_unique[i-1] + 1 :
            
            arr_day[i] = arr_unique[i]
        elif p_cards_value[i] != p_cards_value[i-1] - 1:
            if len(np.where(arr_day > 0)[0]) >= 3:
                arr_id = checkArrInArr(p_cards_value,arr_day)
                arr_id = p_cards[arr_id]
                policy[arr_id] = 1
            arr_day = np.zeros(11)
    return np.where(policy == 1)[0]
@njit()
def getBoDay(state):
    p_cards = state[0:52]
    p_cards = np.where(p_cards == 1)[0]
    p_cards_value = p_cards//4
    policy = np.zeros(52)
    for i in range(52):
        arr_copy = np.copy(p_cards)
        arr_temp = np.where(arr_copy // 4 == i)[0] 
    # check day
    arr_day = np.zeros(11)
    arr_unique = np.unique(p_cards_value)
    for i in range(1,len(arr_unique)):
        if arr_unique[i] == arr_unique[i-1] + 1 :
            arr_day[i] = arr_unique[i]
        elif arr_unique[i] != arr_unique[i-1] - 1:
            if len(np.where(arr_day > 0)[0]) >= 3:
                arr_id = checkArrInArr(p_cards_value,arr_day)
                # print('arr_id',arr_id)
                arr_id = p_cards[arr_id]
                policy[arr_id] = 1
            arr_day = np.zeros(11)
    return np.where(policy == 1)[0]
@njit()
def getHighestCard(action):
  return __ACTIONS__[action][1]

In [ ]:
@njit()
def getTypeAction(action):
  return __ACTIONS__[action][0]
@njit()
def getValueActions(action):
  return __ACTIONS__[action][1]
@njit()
def getLowestValueOfChain(action):
    type_ = getTypeAction(action) - 3
    highestValue = getValueActions(action)
    return highestValue//4 - type_ 
@njit()
def countCardInChain(state,cardValue):
   cards_on_hand = state[0:52]
   cards_on_hand = np.where(cards_on_hand == 1)[0]
  #  print(cards_on_hand//4)
  #  print(cardValue//4)
  #  cards_on_hand_value = cards_on_hand // 4
   count = 0
   for card_value_ in cards_on_hand:
     if  card_value_ // 4 == cardValue // 4 and card_value_ != cardValue:
       count += 1
   return count
@njit()
def getAction(state,actions):
    boDay = getBoDay(state)//4
    for action in actions:
        type_ = getTypeAction(action)
        value_ = getValueActions(action)
        if value_//4 not in boDay and value_//4 != 12:
          # print('value_//4',value_//4)
          return action
        else:
          count_chain = countCardInChain(state,getValueActions(action))
          if count_chain > 2:
            # print('count_chain',count_chain)
            return action
    return -1  
@njit()
def getActionLaDon(state,actions):
    boDay = getBoDay(state)//4
    # check xem co trong bo day va bo doi hay k
    for action in actions:
        type_ = getTypeAction(action)
        value_ = getValueActions(action)
        specialCard = getSpecialCard(state)
        if value_ //4 == 12:
          return action
        if value_//4 not in specialCard//4:
          # print('value_//4',value_//4)
          #  count_chain = countCardInChain(state,getValueActions(action))
          #  if count_chain == 0 or  value_ == 12:
          #   # print('count_chain',count_chain)
            return action
    return -1  
#----------------------------------------------------------------------------------------------------
@njit()
def agentMain(state,per):
  cards_on_hand = state[0:52]
  cards_on_hand = np.where(cards_on_hand == 1)[0]
  actions = getValidActions(state)
  specialCards = getSpecialCard(state)
  boDay = getBoDay(state)
  if np.min(state[107:110]) >=8: 
    if actions[0] == 1 :
        return 0,per
  actions = np.where(actions == 1)[0]
  boDay = getBoDay(state)//4
  type_action = [getTypeAction(action) for action in actions]
  id = np.argmax(np.array(type_action))
# Xu ly danh la 2 ------------------------
  if getTypeAction(actions[id]) == 2 and getValueActions(actions[id])//4 in specialCards//4:
     action_ = getAction(state,actions)
     if action_ != -1:
       return action_,per
     elif np.min(state[107:110]) > 8:
        return actions[0],per
     else :
       return actions[1],per
# Xu ly danh la 1 ----------------------
  if getTypeAction(actions[id]) == 1 and getValueActions(actions[id])//4 in specialCards//4:
      action_ = getActionLaDon(state,actions[id:])
      if action_ != -1 :
        return action_,per
      elif np.min(state[107:110]) > 8:
        return actions[0],per
      else:
        return actions[1],per
  valueCard = __ACTIONS__[actions[id]][1] // 4
  cardsOnHand = (len(cards_on_hand) + np.sum(state[107:110]))/4
  if valueCard > 13/cardsOnHand * 3.33 + 6:
      return actions[0],per
  return actions[id],per

win,per = numba_main_2(agentMain,10000,np.array([0.]),1)
print('win',win)

In [ ]:
0: 5005
1:5038

In [ ]:
def getTypeAction(action):
  return __ACTIONS__[action][0]
def getValueActions(action):
  return __ACTIONS__[action][1]
def getLowestValueOfChain(action):
    type_ = getTypeAction(action) - 3
    highestValue = getValueActions(action)
    return highestValue//4 - type_ 
def countCardInChain(state,cardValue):
   cards_on_hand = state[0:52]
   cards_on_hand = np.where(cards_on_hand == 1)[0]
  #  print(cards_on_hand//4)
  #  print(cardValue//4)
  #  cards_on_hand_value = cards_on_hand // 4
   count = 0
   for card_value_ in cards_on_hand:
     if  card_value_ // 4 == cardValue // 4 and card_value_ != cardValue:
       count += 1
   return count
def getAction(state,actions):
    boDay = getBoDay(state)//4
    for action in actions:
        type_ = getTypeAction(action)
        value_ = getValueActions(action)
        if value_//4 not in boDay and value_//4 != 12:
          # print('value_//4',value_//4)
          return action
        else:
          count_chain = countCardInChain(state,getValueActions(action))
          if count_chain > 2:
            # print('count_chain',count_chain)
            return action
    return -1  
def getActionLaDon(state,actions):
    boDay = getBoDay(state)//4
    # check xem co trong bo day va bo doi hay k
    for action in actions:
        type_ = getTypeAction(action)
        value_ = getValueActions(action)
        specialCard = getSpecialCard(state)
        if value_ //4 == 12:
          return action
        if value_//4 not in specialCard//4:
          # print('value_//4',value_//4)
          #  count_chain = countCardInChain(state,getValueActions(action))
          #  if count_chain == 0 or  value_ == 12:
          #   # print('count_chain',count_chain)
            return action
        
# check xem co trong bo doi hay khong
    return -1  
def getAction3La(state,actions):
   boDay =  getBoDay(state)//4
   # check xem bo 3 co trong bộ dây không
   for action in actions:
        type_ = getTypeAction(action)
        value_ = getValueActions(action)
        specialCard = getSpecialCard(state)
        if value_//4 not in boDay:
           return action
#----------------------------------------------------------------------------------------------------
def agentMain(state,per):
  cards_on_hand = state[0:52]
  cards_on_hand = np.where(cards_on_hand == 1)[0]
  # print('cards_on_hand',cards_on_hand//4)
  actions = getValidActions(state)
  specialCards = getSpecialCard(state)
  boDay = getBoDay(state)
  # print('boDay',boDay//4)
  # print('specialCards',specialCards//4)
  # print('specialCard',specialCards)
  if np.min(state[107:110]) >=8: 
    if actions[0] == 1 :
        return 0,per
  actions = np.where(actions == 1)[0]
  boDay = getBoDay(state)//4
  # print('boDay',boDay)
  type_action = [getTypeAction(action) for action in actions]
  id = np.argmax(type_action)
# Xu ly danh la 2 ------------------------
  if getTypeAction(actions[id]) == 2 and getValueActions(actions[id])//4 in specialCards//4:
     action_ = getAction(state,actions)
     if action_ != -1:
       return action_,per
      #  print('actions[0]',actions[0])
     elif np.min(state[107:110]) > 8:
        return actions[0],per
     else :
       return actions[1],per
# Xu ly danh la 1 ----------------------
  if getTypeAction(actions[id]) == 1 and getValueActions(actions[id])//4 in specialCards//4:
      action_ = getActionLaDon(state,actions[id:])
      if action_ != -1 :
        return action_,per
      elif np.min(state[107:110]) > 8:
        return actions[0],per
      else:
        return actions[1],per
  # Xu ly danh la 3
  # if getTypeAction(actions[id]) == 3:
  #   action_ = getAction3La(state,actions[id:])
  #   if action_ 
  valueCard = __ACTIONS__[actions[id]][1] // 4
  cardsOnHand = (len(cards_on_hand) + np.sum(state[107:110]))/4
  if valueCard > 6.66 + (13-cardsOnHand)*1.18:
      return actions[0],per
  return actions[id],per

win,per = numba_main_2(agentMain,10000,np.array([0.]),1)
print('win',win)

In [ ]:
                                                                                                                                                                                                                                                                                                                                                                        def getTypeAction(action):
  return __ACTIONS__[action][0]
def getValueActions(action):
  return __ACTIONS__[action][1]
def getLowestValueOfChain(action):
    type_ = getTypeAction(action) - 3
    highestValue = getValueActions(action)
    return highestValue//4 - type_ 
def countCardInChain(state,cardValue):
   cards_on_hand = state[0:52]
   cards_on_hand = np.where(cards_on_hand == 1)[0]
  #  print(cards_on_hand//4)
  #  print(cardValue//4)
  #  cards_on_hand_value = cards_on_hand // 4
   count = 0
   for card_value_ in cards_on_hand:
     if  card_value_ // 4 == cardValue // 4 and card_value_ != cardValue:
       count += 1
   return count
def getAction(state,actions):
    boDay = getBoDay(state)//4
    for action in actions:
        type_ = getTypeAction(action)
        value_ = getValueActions(action)
        if value_//4 not in boDay and value_//4 != 12:
          # print('value_//4',value_//4)
          return action
        else:
          count_chain = countCardInChain(state,getValueActions(action))
          if count_chain > 2:
            # print('count_chain',count_chain)
            return action
    return -1  
def getActionLaDon(state,actions):
    boDay = getBoDay(state)//4
    # check xem co trong bo day va bo doi hay k
    for action in actions:
        type_ = getTypeAction(action)
        value_ = getValueActions(action)
        specialCard = getSpecialCard(state)
        if value_ //4 == 12:
          return action
        if value_//4 not in specialCard//4:
          # print('value_//4',value_//4)
          #  count_chain = countCardInChain(state,getValueActions(action))
          #  if count_chain == 0 or  value_ == 12:
          #   # print('count_chain',count_chain)
            return action
        
# check xem co trong bo doi hay khong
    return -1  
def getAction3La(state,actions):
   boDay =  getBoDay(state)//4
   # check xem bo 3 co trong bộ dây không
   for action in actions:
        type_ = getTypeAction(action)
        value_ = getValueActions(action)
        specialCard = getSpecialCard(state)
        if value_//4 not in boDay:
           return action
#----------------------------------------------------------------------------------------------------
def agentMain(state,per):
  cards_on_hand = state[0:52]
  cards_on_hand = np.where(cards_on_hand == 1)[0]
  # print('cards_on_hand',cards_on_hand//4)
  actions = getValidActions(state)
  specialCards = getSpecialCard(state)
  boDay = getBoDay(state)
  # print('boDay',boDay//4)
  # print('specialCards',specialCards//4)
  # print('specialCard',specialCards)
  if np.min(state[107:110]) >=8 and np.max(state[110:113]) == 1: 
    if actions[0] == 1 :
        return 0,per
  actions = np.where(actions == 1)[0]
  boDay = getBoDay(state)//4
  # print('boDay',boDay)
  type_action = [getTypeAction(action) for action in actions]
  id = np.argmax(type_action)
# Xu ly danh la 2 ------------------------
  if getTypeAction(actions[id]) == 2 and getValueActions(actions[id])//4 in specialCards//4:
     action_ = getAction(state,actions)
     if action_ != -1:
       return action_,per
      #  print('actions[0]',actions[0])
     else:
       return actions[1],per
# Xu ly danh la 1 ----------------------
  if getTypeAction(actions[id]) == 1 and getValueActions(actions[id])//4 in specialCards//4:
      action_ = getActionLaDon(state,actions[id:])
      if action_ != -1 :
        return action_,per
      else:
        return actions[1],per
  # Xu ly danh la 3
  # if getTypeAction(actions[id]) == 3:
  #   action_ = getAction3La(state,actions[id:])
  #   if action_ 
  return actions[id],per

win,per = numba_main_2(agentMain,10000,np.array([0.]),1)
print('win',win)                                                                                                                                                                          